In [1]:
import numpy as np

!pip install scikit-learn
from sklearn.model_selection import KFold,cross_val_score,train_test_split
import matplotlib.image as Image
import PIL.Image as Image
import torch
import torch.nn as nn
import torch.nn.functional as f


In [2]:
# m_check = ModelCheckpoint("model/--{epoch:02d}--{val_loss:.4f}.hdf5", monitor = 'val_loss',save_best_only=True)


In [3]:
x = np.load("../train.npy")
x = torch.from_numpy(x)
print(x[0])

y = np.load("y.npy")

y = torch.from_numpy(y)

predx = Image.open("../495.png").resize(((150,112)))
predx= np.asarray(predx).reshape(1,112,150,3)
predx= torch.from_numpy(predx)



tensor([[[1.0000, 1.0000, 1.0000],
         [0.9961, 0.9961, 0.9961],
         [0.9961, 0.9961, 0.9961],
         ...,
         [0.9961, 0.9961, 0.9961],
         [0.9961, 0.9961, 0.9961],
         [1.0000, 1.0000, 1.0000]],

        [[1.0000, 1.0000, 1.0000],
         [0.3647, 0.3647, 0.3647],
         [0.2000, 0.2000, 0.2000],
         ...,
         [0.2000, 0.2000, 0.2000],
         [0.3647, 0.3647, 0.3647],
         [1.0000, 1.0000, 1.0000]],

        [[1.0000, 1.0000, 1.0000],
         [0.1843, 0.1843, 0.1843],
         [0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000],
         [0.1843, 0.1843, 0.1843],
         [1.0000, 1.0000, 1.0000]],

        ...,

        [[1.0000, 1.0000, 1.0000],
         [0.1843, 0.1843, 0.1843],
         [0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000],
         [0.1843, 0.1843, 0.1843],
         [1.0000, 1.0000, 1.0000]],

        [[1.0000, 1.0000, 1.0000],
         [0.3647, 0.3647, 0.3647],
         [0.

<ipython-input-3-8db8159683ad>:11: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  predx= torch.from_numpy(predx)


In [ ]:
class Net(nn.Module):
    def __init__(self):
        supter(Net,self).__init__()
        

In [4]:
def create_model():
    model = Sequential()
    model.add(Conv2D(200,(3,3),padding='same',activation='relu',input_shape=(112,150,3)))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(200,(3,3),padding='same',activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(100,(3,3),padding='same',activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(100,(3,3),padding='same',activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(100,(2,2),padding='same',activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(100,(2,2),padding='same',activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(2,2))

    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(100,activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(100,activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(100,activation="relu"))
    model.add(Dropout(0.3))

    model.add(Dense(100,activation="relu"))
    model.add(Dropout(0.3))

    model.add(Dense(9,activation="softmax"))

    model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['acc'])

    return model


In [5]:
seed = np.random.seed(7)

kf = KFold(n_splits=3, shuffle=True,random_state=seed)


In [6]:

x_train,x_test , y_train,y_test = train_test_split(x,y,shuffle=True , random_state=seed)

In [ ]:
model = create_model()
for train_i,test_i in kf.split(x):
    train_x,train_y = x[train_i],y[train_i]
    test_x, test_y = x[test_i], y[test_i]

    model.fit(train_x,train_y,batch_size=30,epochs=100,validation_split=0.25,callbacks=[m_check])
    score = model.evaluate(test_x,test_y)
    print(score)

Train on 2250 samples, validate on 750 samples
Epoch 1/100


In [ ]:
predy = model.evaluate(x_test,y_test)
print(predy)